* 将 round2 模型里面训练集中预测错误的图片, 和验证集图片找出来 valid_error
     * 加载训练集, 未经过预处理
     * 加载验证集
     * 使用模型进行预测, 得到错误预测的 X_error, y_error 
* 对 valid_error 做数据增强, 放入 /valid_error/label/ 文件夹
* 使用这个数据, 对模型进行第三轮的训练

In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import itertools

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 导入数据

In [2]:
df_train = pd.read_csv('./data/train/Annotations/label.csv')
df_train.columns = ['image_id', 'class', 'label']
df_train.head()

,image_id,class,label
0,Images/collar_design_labels/0ef580b4deabcd9fa4...,collar_design_labels,ynnnn
1,Images/collar_design_labels/87ccc33937821a97ad...,collar_design_labels,nnynn
2,Images/collar_design_labels/26caac7d1f1b36fb9d...,collar_design_labels,ynnnn
3,Images/collar_design_labels/396ab4e7cbc6798100...,collar_design_labels,nynnn
4,Images/collar_design_labels/fc4a679c2bf209de13...,collar_design_labels,ynnnn


In [3]:
classes = ['collar_design_labels', 'neckline_design_labels', 'skirt_length_labels',
          'sleeve_length_labels', 'neck_design_labels', 'coat_length_labels', 'lapel_design_labels',
          'pant_length_labels']

## 加载数据

In [4]:
cur_class = classes[3]
prefix_cls = cur_class.split('_')[0]

df_load = df_train[(df_train['class'] == cur_class)].copy()
df_load.reset_index(inplace=True)
del df_load['index']

print('{0}: {1}'.format(cur_class, len(df_load)))
df_load.head()

sleeve_length_labels: 17285


,image_id,class,label
0,Images/sleeve_length_labels/f5c414bda8a9bb97f6...,sleeve_length_labels,nnnnnnmym
1,Images/sleeve_length_labels/ffeef43a34d68b2a47...,sleeve_length_labels,nnynnnnnn
2,Images/sleeve_length_labels/4be61ca727ad25645e...,sleeve_length_labels,nnnnnnmmy
3,Images/sleeve_length_labels/b9d69a26db06295bfa...,sleeve_length_labels,nnnynnnnn
4,Images/sleeve_length_labels/fb7c3a0181d538b52e...,sleeve_length_labels,nnnnnnynn


In [5]:
n_ration=0.9
valid_data=df_load[int(n_ration*len(df_load)):]
df_load=df_load[:int(n_ration*len(df_load))]

## 加载训练集

In [6]:
n = len(df_load)
n_class = len(df_load['label'][0])
width = 299 # 定义图片大小

In [7]:
X_train = np.zeros((n, width, width, 3), dtype=np.uint8)
y_train = np.zeros((n, n_class), dtype=np.uint8)

In [8]:
for i in tqdm(range(n)):
    tmp_label = df_load['label'][i]
    X_train[i] = cv2.resize(cv2.imread('./data/train/{0}'.format(df_load['image_id'][i])), (width, width))
    y_train[i][tmp_label.find('y')] = 1

100%|██████████| 15556/15556 [01:31<00:00, 170.49it/s]


## 加载验证集

In [9]:
# 重置索引
valid_data = valid_data.reset_index(drop=True)

In [10]:
n = len(valid_data)
n_class = len(valid_data['label'][0])
width = 299 # 定义图片大小

In [11]:
X_valid = np.zeros((n, width, width, 3), dtype=np.uint8)
y_valid = np.zeros((n, n_class), dtype=np.uint8)

In [12]:
for i in tqdm(range(n)):
    tmp_label = valid_data['label'][i]
    X_valid[i] = cv2.resize(cv2.imread('./data/train/{0}'.format(valid_data['image_id'][i])), (width, width))
    y_valid[i][tmp_label.find('y')] = 1

100%|██████████| 1729/1729 [00:09<00:00, 173.49it/s]


In [13]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((15556, 299, 299, 3), (15556, 9), (1729, 299, 299, 3), (1729, 9))

## 建立模型

In [14]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import *
from keras.applications.inception_v3 import preprocess_input

Using TensorFlow backend.


In [15]:
cnn_model = InceptionResNetV2(include_top=False, input_shape=(width, width, 3), weights='imagenet')
inputs = Input((width, width, 3))

x = inputs
x = Lambda(preprocess_input, name='preprocessing')(x)
x = cnn_model(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(n_class, activation='softmax', name='softmax')(x)

model = Model(inputs, x)

In [16]:
adam = Adam(lr=0.001)
prefix_cls = cur_class.split('_')[0]

In [17]:
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model.load_weights('./models/{}.best0516b_InceptionResNetV2_round2.h5'.format(prefix_cls))

In [20]:
# Predict the values from the validation dataset
Y_pred = model.predict(X_train)

# Convert predictions classes to one hot vectors
Y_pred_classes = np.argmax(Y_pred, axis=1)

# Convert validation observations to one hot vectors
Y_true = np.argmax(y_train, axis=1)

# Display some error results
# Errors are difference between predicted labels and true labels
errors = (Y_pred_classes - Y_true != 0)

In [ ]:
Y_pred_classes_errors = Y_pred_classes[errors]
Y_pred_errors = Y_pred[errors]
Y_true_errors = Y_true[errors]
X_val_errors = X_train[errors]

In [ ]:
nrows = 2
ncols = 2
def display_errors(errors_index, img_errors, pred_errors, obs_errors):
    """This function shows 6 images with their predicted and real labels"""
    n = 0
    
    fig, ax = plt.subplots(nrows, ncols, sharex=True, sharey=True)
    for row in range(nrows):
        for col in range(ncols):
            error = errors_index[n]
            ax[row, col].imshow((img_errors[error]).reshape((299, 299, 3)))
            ax[row, col].set_title("Predicted label :{}\nTrue label :{}".format(pred_errors[error], obs_errors[error]))
            n += 1

In [ ]:
#Probabilities of the wrong predicted numbers
Y_pred_errors_prob = np.max(Y_pred_errors, axis=1)

# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(Y_pred_errors, Y_true_errors, axis=1))

# Difference between the probability of the predicted label and the true label
delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors

# Sorted list of the delta prob errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)

# Top 6 errors
most_imaportant_errors = sorted_dela_errors[-nrows*ncols:]

# Show the top 6 errors
display_errors(most_imaportant_errors, X_val_errors, Y_pred_classes_errors, Y_true_errors)

## 找出错误图片编号, 并与 valid 合并

In [21]:
df_errors = df_load[errors]

df_temp = pd.concat([df_errors, valid_data])

# 重置索引
df_temp = df_temp.reset_index(drop=True)

df_temp.to_csv('./valid_error/{}_valid_errors.csv'.format(prefix_cls), header=None, index=False)

## 对 valid_error 做数据增强

In [22]:
prefix_cls = cur_class.split('_')[0]

In [23]:
df_temp = pd.read_csv('./valid_error/{}_valid_errors.csv'.format(prefix_cls))
df_temp.columns = ['image_id', 'class', 'label']
df_temp.head()

,image_id,class,label
0,Images/sleeve_length_labels/4be61ca727ad25645e...,sleeve_length_labels,nnnnnnmmy
1,Images/sleeve_length_labels/28c823bbcd13f81e7f...,sleeve_length_labels,nnnnmynnn
2,Images/sleeve_length_labels/4480385f3cfb87a3fd...,sleeve_length_labels,nnnnnnmmy
3,Images/sleeve_length_labels/25294867fc2cf4fdd7...,sleeve_length_labels,nnnnnnnny
4,Images/sleeve_length_labels/880fa1a43b0496eb5c...,sleeve_length_labels,nnnnnnmmy


In [24]:
from skimage import data, img_as_float
from skimage import exposure

In [25]:
import os

def mkdir_if_not_exist(path):
    if not os.path.exists(os.path.join(*path)):
        os.makedirs(os.path.join(*path))
        
mkdir_if_not_exist(['./data/valid_error/Images/{}/'.format(cur_class)])

In [26]:
n = len(df_temp)

In [27]:
rootdir = './data/train/'
outdir = './data/valid_error/Images/{}/'.format(cur_class)
m = 0
for i in range(n):
    currentPath = rootdir + df_temp['image_id'][i]
    filename = df_temp['image_id'][i].split('/')[-1]
    
    # 先填充, 再缩放
    img = cv2.imread(currentPath)
    dst0 = cv2.resize(img, (299, 299), interpolation=cv2.INTER_AREA)
    rows,cols,channels = dst0.shape
    
    # 水平翻转
    dst1 = cv2.flip(dst0, 1)
    
#     # 高斯噪声
#     dst0.astype("float")
#     Gauss_noise = np.random.normal(0, 20, (rows, cols, channels))
#     dst2 = dst0 + Gauss_noise
#     dst2 = np.where(dst2 < 0, 0, np.where(dst2 > 255, 255, dst2))
    
#     # 顺时针旋转 5°/ 放大 5%
#     M1 = cv2.getRotationMatrix2D((cols/2,rows/2),5,1) 
#     M2 = cv2.getRotationMatrix2D((cols/2,rows/2),0,1.05) 
#     dst3 = cv2.warpAffine(dst0,M1,(cols,rows))
#     dst4 = cv2.warpAffine(dst0,M2,(cols,rows))
    
#     # 直方图
#     dst5 = exposure.equalize_hist(dst0)*255
    
    cv2.imwrite(outdir + filename + "-0.jpg", dst0)
    cv2.imwrite(outdir + filename + "-1.jpg", dst1)
#     cv2.imwrite(outdir + filename + "-2.jpg", dst2.astype("uint8"))
#     cv2.imwrite(outdir + filename + "-3.jpg", dst3)
#     cv2.imwrite(outdir + filename + "-4.jpg", dst4)
#     cv2.imwrite(outdir + filename + "-5.jpg", dst5)  
    
    m += 1
    if m%100 == 0:
        print('{} images have preprocessed.'.format(m))

100 images have preprocessed.
200 images have preprocessed.
300 images have preprocessed.
400 images have preprocessed.
500 images have preprocessed.
600 images have preprocessed.
700 images have preprocessed.
800 images have preprocessed.
900 images have preprocessed.
1000 images have preprocessed.
1100 images have preprocessed.
1200 images have preprocessed.
1300 images have preprocessed.
1400 images have preprocessed.
1500 images have preprocessed.
1600 images have preprocessed.
1700 images have preprocessed.
1800 images have preprocessed.
1900 images have preprocessed.
2000 images have preprocessed.
2100 images have preprocessed.
2200 images have preprocessed.
2300 images have preprocessed.
2400 images have preprocessed.
2500 images have preprocessed.
2600 images have preprocessed.
2700 images have preprocessed.
2800 images have preprocessed.
2900 images have preprocessed.
3000 images have preprocessed.
3100 images have preprocessed.
3200 images have preprocessed.
3300 images have 

## 加载训练集

In [28]:
n = len(df_temp)
n_class = len(df_temp['label'][0])
width = 299 # 定义图片大小

In [29]:
X_train = np.zeros((n*2, width, width, 3), dtype=np.uint8)
y_train = np.zeros((n*2, n_class), dtype=np.uint8)

In [30]:
for i in tqdm(range(n)):
    tmp_label = df_temp['label'][i]
    for j in range(2):
        X_train[i*2+j] = cv2.resize(cv2.imread('./data/valid_error/{0}'.format(df_temp['image_id'][i] + "-%s.jpg"%j)), (width, width))
        y_train[i*2+j][tmp_label.find('y')] = 1

100%|██████████| 4215/4215 [00:10<00:00, 406.26it/s]


## 建立模型

In [31]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import *
from keras.applications.inception_v3 import preprocess_input

In [32]:
cnn_model = InceptionResNetV2(include_top=False, input_shape=(width, width, 3), weights='imagenet')
inputs = Input((width, width, 3))

x = inputs
x = Lambda(preprocess_input, name='preprocessing')(x)
x = cnn_model(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(n_class, activation='softmax', name='softmax')(x)

model = Model(inputs, x)

In [33]:
adam = Adam(lr=0.00005)
prefix_cls = cur_class.split('_')[0]

In [34]:
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
# model.load_weights('./models/{}.best0516b_InceptionResNetV2_round2.h5'.format(prefix_cls))

In [35]:
checkpointer = ModelCheckpoint(filepath='./models/{0}.best0517a_InceptionResNetV2_round3.h5'.format(prefix_cls), verbose=1, 
                               save_best_only=True)

try:
    h = model.fit(X_train, y_train, batch_size=32, epochs=2, 
                  callbacks=[EarlyStopping(patience=3), checkpointer], 
                  shuffle=True, 
                  validation_data=(X_valid, y_valid))
except KeyboardInterrupt:
    print('KeyboardInterrupt')

Train on 8430 samples, validate on 1729 samples
Epoch 1/2
8430/8430 [==============================] - 340s 40ms/step - loss: 1.6386 - acc: 0.3539 - val_loss: 0.9652 - val_acc: 0.6669

Epoch 00001: val_loss improved from inf to 0.96524, saving model to ./models/sleeve.best0517a_InceptionResNetV2_round3.h5
Epoch 2/2
8430/8430 [==============================] - 291s 35ms/step - loss: 0.8224 - acc: 0.6943 - val_loss: 0.4495 - val_acc: 0.8809

Epoch 00002: val_loss improved from 0.96524 to 0.44954, saving model to ./models/sleeve.best0517a_InceptionResNetV2_round3.h5


In [62]:
model.evaluate(x=X_valid, y=y_valid, batch_size=None, verbose=1, sample_weight=None, steps=None)

906/906 [==============================] - 9s 10ms/step


[0.005291031291153258, 1.0]

## 预处理图片进行全部预测

In [66]:
mkdir_if_not_exist(['./data/test-preprocessing/Images/{}/'.format(cur_class)])

In [67]:
rootdir = './data/week-rank/Images/{}'.format(cur_class)  # 指明被遍历的文件夹
m = 0
outdir = './data/test-preprocessing/Images/{}/'.format(cur_class)

for parent, dirnames, filenames in os.walk(rootdir):
    for filename in filenames:
        currentPath = os.path.join(parent, filename)

        # 先填充, 再缩放
        img = cv2.imread(currentPath)
        dst0 = cv2.resize(img, (299, 299), interpolation=cv2.INTER_AREA)
        rows,cols,channels = dst0.shape

#         # 顺时针旋转 5°/ 放大 5%
#         M1 = cv2.getRotationMatrix2D((cols/2,rows/2),5,1) 
#         M2 = cv2.getRotationMatrix2D((cols/2,rows/2),0,1.05) 
#         dst1 = cv2.warpAffine(dst0,M1,(cols,rows))
#         dst2 = cv2.warpAffine(dst0,M2,(cols,rows))
        
#         # 向下平移 0.2
#         M3 = np.float32([[1,0,0],[0,1,int(cols*0.2)]])
#         dst3 = cv2.warpAffine(dst0,M3,(cols,rows))
        
        # 水平翻转
        dst1 = cv2.flip(dst0, 1)
        
        # 高斯噪声
        dst0.astype("float")
        Gauss_noise = np.random.normal(0, 20, (rows, cols, channels))
        dst2 = dst0 + Gauss_noise
        dst2 = np.where(dst2 < 0, 0, np.where(dst2 > 255, 255, dst2))
        
#         # 对翻转后图片, 左右旋转 20°
#         M4 = cv2.getRotationMatrix2D((cols/2,rows/2),20,1)
#         M5 = cv2.getRotationMatrix2D((cols/2,rows/2),-20,1)
#         dst5 = cv2.warpAffine(dst4,M4,(cols,rows))
#         dst6 = cv2.warpAffine(dst4,M5,(cols,rows))
        
#         # 翻转后向下平移 0.2
#         M6 = np.float32([[1,0,0],[0,1,int(cols*0.2)]])
#         dst7 = cv2.warpAffine(dst4,M6,(cols,rows))
        
        cv2.imwrite(outdir + filename + "-0.jpg", dst0)
        cv2.imwrite(outdir + filename + "-1.jpg", dst1)
        cv2.imwrite(outdir + filename + "-2.jpg", dst2.astype("uint8"))
#         cv2.imwrite(outdir + filename + "-3.jpg", dst3)
#         cv2.imwrite(outdir + filename + "-4.jpg", dst4.astype("uint8"))
#         cv2.imwrite(outdir + filename + "-5.jpg", dst5)
#         cv2.imwrite(outdir + filename + "-6.jpg", dst6)
#         cv2.imwrite(outdir + filename + "-7.jpg", dst7)
        m += 1
        if m%100 == 0:
            print('{} images have preprocessed.'.format(m))

100 images have preprocessed.
200 images have preprocessed.
300 images have preprocessed.
400 images have preprocessed.
500 images have preprocessed.
600 images have preprocessed.
700 images have preprocessed.
800 images have preprocessed.
900 images have preprocessed.
1000 images have preprocessed.
1100 images have preprocessed.
1200 images have preprocessed.
1300 images have preprocessed.
1400 images have preprocessed.
1500 images have preprocessed.


In [36]:
df_test = pd.read_csv('./data/week-rank/Tests/question.csv', header=None)
df_test.columns = ['image_id', 'class', 'x']
del df_test['x']
df_test.head()

,image_id,class
0,Images/collar_design_labels/1078bc9a4fa91aef69...,collar_design_labels
1,Images/collar_design_labels/272c5d069581efc94f...,collar_design_labels
2,Images/collar_design_labels/2e67922d82216cc756...,collar_design_labels
3,Images/collar_design_labels/d9726ceee17c7ee811...,collar_design_labels
4,Images/collar_design_labels/1b7de7f9def99a5aa6...,collar_design_labels


In [37]:
df_load2 = df_test[(df_test['class'] == cur_class)].copy()
df_load2.reset_index(inplace=True)
del df_load2['index']

print('{0}: {1}'.format(cur_class, len(df_load2)))
df_load2.head()

sleeve_length_labels: 2895


,image_id,class
0,Images/sleeve_length_labels/3a59bdc48834d4578e...,sleeve_length_labels
1,Images/sleeve_length_labels/a12e001f27c07365e6...,sleeve_length_labels
2,Images/sleeve_length_labels/ba5c56b7bc61711db3...,sleeve_length_labels
3,Images/sleeve_length_labels/efd82f9d354c0b2d53...,sleeve_length_labels
4,Images/sleeve_length_labels/1283a0ff4560bba965...,sleeve_length_labels


In [ ]:
n = len(df_load2)
width = 299 # 定义图片大小
X_test = np.zeros((n*3, width, width, 3), dtype=np.uint8)

for i in tqdm(range(n)):
    for j in range(3):
        X_test[i*3+j] = cv2.resize(cv2.imread('./data/test-preprocessing/{0}'.format(df_load2['image_id'][i] + "-%s.jpg"%j)), (width, width))

 97%|█████████▋| 2803/2895 [00:25<00:00, 109.76it/s]

In [ ]:
# model.load_weights('./models/{}.best0515a_InceptionResNetV2.h5'.format(prefix_cls))

In [ ]:
test_np = model.predict(X_test, batch_size=256)

In [ ]:
# 取同一张图片不同预处理方式的平均值
result = []

for i, row in df_load2.iterrows():
    tmp_list = (test_np[i*3+0] + test_np[i*3+1] + test_np[i*3+2])/3
    tmp_result = ''
    for tmp_ret in tmp_list:
        tmp_result += '{:.4f};'.format(tmp_ret)
        
    result.append(tmp_result[:-1])

df_load2['result'] = result
df_load2.head()

In [ ]:
df_load2.to_csv('./result/{}_0517b_InceptionResNetV2.csv'.format(prefix_cls), header=None, index=False)